# Hanbury-Brown-Twiss effect



We try to simulate noise related to purity and brightness. 

In [ ]:
import perceval as pcvl
import perceval.components.unitary_components as comp
print(pcvl.__version__)
import numpy as np
import math
import sympy as sp

st1,st2,st3=pcvl.BasicState([0,1,1,0]),pcvl.BasicState([1,0,1,0]),pcvl.BasicState([1,0,0,1])
State=pcvl.StateVector.normalize(3*st1+2*np.sqrt(2)*st2+2*np.sqrt(2)*st3)
#pcvl.StateVector.normalize(State)
#state=(3*pcvl.BasicState([0,1,1,0]))
#print(state)
print(State)

: 

We calculate $g^{(2)}(0)=\frac{\braket{\Psi,0|\hat{n}_{3}\hat{n}_{4}|\Psi,0}}{\braket{\Psi,0|\hat{n}_{3}|\Psi,0}\braket{\Psi,0|\hat{n}_{4}|\Psi,0}}$

In [ ]:
def g20_after_circuit(input_state):
    numerator=0
    denominator1=0
    denominator2=0
    circuit=pcvl.Circuit(2)
    circuit.add((0,1),comp.BS())
    slos_backend = pcvl.BackendFactory().get_backend("SLOS")
    s = slos_backend(circuit.U,use_symbolic=False)
    for o, p in s.allstateprob_iterator(input_state):
        numerator+=p*o[0]*o[1]
        denominator1+=p*o[0]
        denominator2+=p*o[1]
    return numerator/(denominator1*denominator2)

: 

We calculate a coherent state distribution $$\ket{\alpha}$$ following a Poisson law of parameter $$\alpha$$

In [ ]:
def c(alpha,n):
    return alpha**(n) /math.sqrt(math.factorial(n))

def S(alpha,n):
    e=np.exp(-0.5*alpha**2)
    return np.sqrt(sum([(e*c(alpha,i))**2 for i in range(n)]))
    
def coherent_state(alpha,N):
    e=np.exp(-0.5*alpha**2)
    state =e* pcvl.StateVector([0,0])
    for n in range(1,N):
        state=S(alpha,n)*state + e*c(alpha,n) * pcvl.StateVector([n,0])
        #print(S(alpha,n))
    return state

def coherent_state_2(alpha,N):
    e=np.exp(-0.5*alpha**2)
    state =e* pcvl.StateVector([0,0])
    for n in range(1,N):
        state=S(alpha,n)*state + e*c(alpha,n) * pcvl.StateVector([n,0])
        #print(S(alpha,n))
    return state

def any_state(f,parameters,N):
    s=sum([f(parameters,n) for n in range(N)])
    state =(1/s)* pcvl.StateVector([0,0])
    for n in range(1,N):
        S=np.sqrt(sum([((1/s)*f(parameters,i))**2 for i in range(n)]))
        state=S*state + (1/s)*f(parameters,n) * pcvl.StateVector([n,0])
        #print(S(alpha,n))
    return state
print(coherent_state(2,40))
print(any_state(c,2,40))

: 

In [ ]:
def bruit_gaussien(mu,sigma,N):
    b=np.random.normal(mu,sigma,size=N)
    s=np.sqrt(sum([b[i]**2 for i in range(N)]))
    state=(1/s)*b[0]*pcvl.StateVector([0,0])
    for n in range(1,N):
        S=np.sqrt(sum([b[i]**2 for i in range(n)])/s)
        state=S*state +(1/s)*b[n]*pcvl.StateVector([n,0])
    return state

print(bruit_gaussien(0,1,40))

: 

In [ ]:
def bruit_gaussien_2(mu,sigma,N):
    b=np.random.normal(mu,sigma,size=N)
    for i in range(N):
        b[i]=abs(b[i])
    state=(1/np.sqrt(N))*pcvl.StateVector([int(b[0]),0])
    for n in range(1,N):
        state=np.sqrt(n)*state+(1/np.sqrt(N))*pcvl.StateVector([int(b[n]),0])
    return state

: 

**Crée un état selon une distribution arbitraire**

sum_coefs(coefs,n) calcule $\sqrt{ \sum_{k=0}^{n-1} coefs[k] }$

In [ ]:
def sum_coefs(coefs,n):
    return math.sqrt(sum([coefs[k]**2 for k in range(n)]))

def state_distrib(coefs):
    norme_tot = sum_coefs(coefs,len(coefs))
    state = pcvl.StateVector([0])
    for n,c in enumerate(coefs[1:]):
        state = sum_coefs(coefs,n+1)/norme_tot*state + c/norme_tot*pcvl.StateVector([n+1])
    return state

: 

Test de la fonction state_distrib :

In [ ]:
coefs = [1/2**(k/2) for k in range(1,20)]
#print(coefs)

X = state_distrib(coefs)
print(X)

: 

In [ ]:
input_state = pcvl.BasicState("|1,1>")
N=40
alpha=2
print(coherent_state(alpha,N))
print(g20_after_circuit(input_state))
print(g20_after_circuit(coherent_state(alpha,N)))
state=5*pcvl.StateVector([1,1])+coherent_state(np.sqrt(4),40)
print(g20_after_circuit(state))

: 

In [ ]:
input_state = pcvl.BasicState("|1,1>")
N=6
sigma=1
b=bruit_gaussien(0,sigma,N)
print(b)
print(g20_after_circuit(input_state))
print(g20_after_circuit(b))
state=10*pcvl.StateVector([1,1])+b
print(g20_after_circuit(state))

: 

In [ ]:
input_state = pcvl.BasicState("|1,1>")
N=50
sigma=2
b=bruit_gaussien_2(0,sigma,N)
print(b)
print(g20_after_circuit(input_state))
print(g20_after_circuit(b))
state=10*pcvl.StateVector([1,1])+b
print(g20_after_circuit(state))

: 

In [ ]:
#help(pcvl.Source)
brightness,purity=1,1
#help(pcvl.Processor)
source=pcvl.Source(brightness,purity)
p = pcvl.Processor("Naive", comp.BS(),source)
pcvl.pdisplay(p)
states = {
    pcvl.BasicState([1, 0, 1, 0]): "00",
    pcvl.BasicState([1, 0, 0, 1]): "01",
    pcvl.BasicState([0, 1, 1, 0]): "10",
    pcvl.BasicState([0, 1, 0, 1]): "11"
}

ca = pcvl.algorithm.Analyzer(p, states)
ca.compute(expected={"00": "00", "01": "01", "10": "11", "11": "10"})
pcvl.pdisplay(ca)
print("performance=%s, error rate=%.3f%%" % (pcvl.simple_float(ca.performance)[1], ca.error_rate * 100))

: 